# HPECPAPI (Simplify version)

#### If you are in hurry --> click [this](#Code-Snip)

## Getting Session ID

### Minimum request

In [19]:
curl -k -i -s --request POST "http://your-gateway-ip.hpeilab.com:8080/api/v2/session" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"name": "username",
"password": "password",
"tenant_name": "test-tenant"
}'

HTTP/1.1 201 Created
Access-Control-Allow-Origin: *
Content-Length: 13
Content-Type: text/plain
Date: Fri, 30 Apr 2021 08:45:41 GMT
Location: /api/v2/session/0eb6b7bd-a990-4518-9fff-aa7f6b74e957
Server: HPE Ezmeral Container Platform 5.3

201 Created


### Request with variable

In [22]:
#
# environment variables to be adjusted/verified by the student
#
username="username" # your HPE ECP tenant login credentials - username
password="password" # your HPE ECP tenant login credentials - password - 
#
# fixed environment variables setup by the HPE ECP Lab administrator - please DO NOT MODIFY!!
#
HTTPs="http"
controller_endpoint="your-gateway-ip:8080"
gateway_host="your-gateway-ip"
tenantname="test-tenant"
tenantlowername=$(echo $tenantname | tr '[A-Z]' '[a-z]')

echo "your working context is:" $username-$password-$tenantname 

your working context is: hpecpcli-hpecpcli-test-tenant


In [23]:
curl -k -i -s --request POST "${HTTPs}://${controller_endpoint}/api/v2/session" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"name": "'"$username"'",
"password": "'"$password"'",
"tenant_name": "'"$tenantname"'"
}'

HTTP/1.1 201 Created
Access-Control-Allow-Origin: *
Content-Length: 13
Content-Type: text/plain
Date: Fri, 30 Apr 2021 08:46:00 GMT
Location: /api/v2/session/acc6e2a3-1795-4acc-a476-09f19a40affb
Server: HPE Ezmeral Container Platform 5.3

201 Created


## optaining the session location

In [24]:
sessionlocation=$(curl -k -i -s --request POST "${HTTPs}://${controller_endpoint}/api/v2/session" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"name": "'"$username"'",
"password": "'"$password"'",
"tenant_name": "'"$tenantname"'"
}' | grep Location | awk '{print $2}' | tr -d '\r') #we remove any cr that might exist

echo $sessionlocation

/api/v2/session/69eacb50-b046-40ec-ab14-69962184b2d5


In [13]:
tenantname="test-tenant" #case sensitive
 
echo "your operation context is:" $username "on tenant" $tenantname
echo "your REST API endpoint is:" $controller_endpoint

sessionlocation=$(curl -k -i -s --request POST "http://${controller_endpoint}/api/v2/session" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"name": "'"$username"'",
"password": "'"$password"'",
"tenant_name": "'"$tenantname"'"
}' | grep Location | awk '{print $2}' | tr -d '\r') #we remove any cr that might exist
echo "This is your session location: " $sessionlocation
SessionId=$(echo $sessionlocation | cut -d'/' -f 5) # extract sessionId for later, for logout
echo "This is your session_Id:" $SessionId

your operation context is: hpecpcli on tenant test-tenant
your REST API endpoint is: ez53-gateway.hpeilab.com:8080
This is your session location:  /api/v2/session/641de4c9-f7b4-4e5f-85d2-26917334e7b2
This is your session_Id: 641de4c9-f7b4-4e5f-85d2-26917334e7b2


## Getting the KUBECONFIG file

download the kubeconfig file to here

In [15]:
curl -k -s --request GET "http://${controller_endpoint}/api/v2/k8skubeconfig" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' > ./kubeconfig

assign the kubeconfig value to use that k8s session

In [25]:
#define the Kubeconfig file as a shell environment variable to tell kubectl where to look for the kubeconfig file
export KUBECONFIG="./kubeconfig"
echo $KUBECONFIG
# kubectl config view

./kubeconfig


In [26]:
kubectl get pod

NAME           READY   STATUS    RESTARTS   AGE
kdss-qmwn5-0   1/1     Running   0          9d


#### Closing your login session

In [9]:
curl -k -i -s --request DELETE "http://${controller_endpoint}/api/v2/session/${SessionId}" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json'

HTTP/1.1 204 No Content
Access-Control-Allow-Origin: *
Content-Length: 0
Content-Type: text/plain
Date: Fri, 30 Apr 2021 09:19:04 GMT
Server: HPE Ezmeral Container Platform 5.3



In [10]:
kubectl get pods

error: You must be logged in to the server (Unauthorized)


: 1

# Code Snip

In [1]:
#
# environment variables to be adjusted/verified by the student
#
username="username" # your HPE ECP tenant login credentials - username
password="ipassword" # your HPE ECP tenant login credentials - password
tenantname="test-tenant"
#
# fixed environment variables setup by the HPE ECP Lab administrator
#
HTTPs="http"
controller_endpoint="ez53-gateway.hpeilab.com:8080"
gateway_host="ez53-gateway.hpeilab.com"


tenantlowername=$(echo $tenantname | tr '[A-Z]' '[a-z]')

echo "your working context is:" $username-$password-$tenantname 
echo "your operation context is:" $username "on tenant" $tenantname
echo "your REST API endpoint is:" $controller_endpoint

your working context is: hpecpcli-hpecpcli-test-tenant
your operation context is: hpecpcli on tenant test-tenant
your REST API endpoint is: ez53-gateway.hpeilab.com:8080


In [2]:
sessionlocation=$(curl -k -i -s --request POST "${HTTPs}://${controller_endpoint}/api/v2/session" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"name": "'"$username"'",
"password": "'"$password"'",
"tenant_name": "'"$tenantname"'"
}' | grep Location | awk '{print $2}' | tr -d '\r') #we remove any cr that might exist
echo "This is your session location: " $sessionlocation
SessionId=$(echo $sessionlocation | cut -d'/' -f 5) # extract sessionId for later, for logout
echo "This is your session_Id:" $SessionId

This is your session location:  /api/v2/session/4b163c1f-d890-4f5a-8c2a-40dc33847ce2
This is your session_Id: 4b163c1f-d890-4f5a-8c2a-40dc33847ce2


In [5]:
curl -k -s --request GET "http://${controller_endpoint}/api/v2/k8skubeconfig" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' > ./kubeconfig

export KUBECONFIG="./kubeconfig"

In [6]:
kubectl get pods

NAME           READY   STATUS    RESTARTS   AGE
kdss-qmwn5-0   1/1     Running   0          9d


## Code (one-line)

In [1]:
curl -k -s --request GET "http://ez53-gateway.hpeilab.com:8080/api/v2/k8skubeconfig" \
--header "X-BDS-SESSION: $(curl -k -i -s --request POST "http://ez53-gateway.hpeilab.com:8080/api/v2/session" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"name": "change-your-user-name",
"password": "change-your-user-password",
"tenant_name": "change-the-tenant-you-want"
}' | grep Location | awk '{print $2}' | tr -d '\r')" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' > ./kubeconfig

export KUBECONFIG="./kubeconfig"

In [2]:
kubectl get pods

NAME           READY   STATUS    RESTARTS   AGE
kdss-qmwn5-0   1/1     Running   0          9d
